In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression

# Class, for use in pipelines, to select certain columns from a DataFrame and convert to a numpy array
# From A. Geron: Hands-On Machine Learning with Scikit-Learn & TensorFlow, O'Reilly, 2017
# Modified by Derek Bridge to allow for casting in the same ways as pandas.DataFrame.astype
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names, dtype=None):
        self.attribute_names = attribute_names
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_selected = X[self.attribute_names]
        if self.dtype:
            return X_selected.astype(self.dtype).values
        return X_selected.values
    
# Class, for use in pipelines, to binarize nominal-valued features (while avoiding the dummy variabe trap)
# By Derek Bridge, 2017
class FeatureBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, features_values):
        self.features_values = features_values
        self.num_features = len(features_values)
        self.labelencodings = [LabelEncoder().fit(feature_values) for feature_values in features_values]
        self.onehotencoder = OneHotEncoder(sparse=False,
            n_values=[len(feature_values) for feature_values in features_values])
        self.last_indexes = np.cumsum([len(feature_values) - 1 for feature_values in self.features_values])
    def fit(self, X, y=None):
        for i in range(0, self.num_features):
            X[:, i] = self.labelencodings[i].transform(X[:, i])
        return self.onehotencoder.fit(X)
    def transform(self, X, y=None):
        for i in range(0, self.num_features):
            X[:, i] = self.labelencodings[i].transform(X[:, i])
        onehotencoded = self.onehotencoder.transform(X)
        return np.delete(onehotencoded, self.last_indexes, axis=1)
    def fit_transform(self, X, y=None):
        onehotencoded = self.fit(X).transform(X)
        return np.delete(onehotencoded, self.last_indexes, axis=1)
    def get_params(self, deep=True):
        return {"features_values" : self.features_values}
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            self.setattr(parameter, value)
        return self

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
def get_nan_values_for_column(df, column_name):
    return df[column_name].isna().sum()

In [7]:
def drop_nan_values_from_columns(df, column_names_list):
    '''
    This function removes all rows, where at least one column a row,
    from the list 'column_names_list', contains a NaN value
    '''
    df_tmp = df.dropna(subset=column_names_list)
    df_tmp.reset_index(drop=True, inplace=True)
    
    return df_tmp

In [8]:
df = pd.read_csv('companies.csv', low_memory=False)
df

,acquisition_id,relationship_object_id,price_amount,price_currency_code,acquired_at,full_name,subject,Aquired,entity_type,entity_id,normalized_name,category_code,status,founded_at,closed_at,homepage_url,country_code,state_code_x,city_x,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,relationships,description_y,city_y,zip_code,latitude,longitude
0,209,c:59,3.100000e+09,USD,2008-03-11,Brian Kennish,NaN,False,Company,59,google,search,ipo,1998-09-07,NaN,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,29.0,26.0,1998-08-01,1999-06-07,2.0,25100000.0,1046.0,Google Mexico,Mexico,1100,19.507519,-99.030156
1,209,c:59,3.100000e+09,USD,2008-03-11,Brian Kennish,NaN,False,Company,59,google,search,ipo,1998-09-07,NaN,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,29.0,26.0,1998-08-01,1999-06-07,2.0,25100000.0,1046.0,Google Haifa,Haifa,31905,NaN,NaN
2,209,c:59,3.100000e+09,USD,2008-03-11,Brian Kennish,NaN,False,Company,59,google,search,ipo,1998-09-07,NaN,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,29.0,26.0,1998-08-01,1999-06-07,2.0,25100000.0,1046.0,Google Tel Aviv,Tel-Aviv,66183,NaN,NaN
3,209,c:59,3.100000e+09,USD,2008-03-11,Brian Kennish,NaN,False,Company,59,google,search,ipo,1998-09-07,NaN,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,29.0,26.0,1998-08-01,1999-06-07,2.0,25100000.0,1046.0,Google Istanbul,Istanbul,34394,NaN,NaN
4,209,c:59,3.100000e+09,USD,2008-03-11,Brian Kennish,NaN,False,Company,59,google,search,ipo,1998-09-07,NaN,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,29.0,26.0,1998-08-01,1999-06-07,2.0,25100000.0,1046.0,Google Dubai,Dubai,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7210973,4521,c:3246,NaN,USD,2006-09-06,Michael Klett,Computer Engineering,False,Company,3246,ibm,software,ipo,NaN,NaN,http://www.ibm.com,USA,NY,Armonk,New York,2000-08-28,2008-03-25,2.0,2.0,NaN,NaN,NaN,NaN,870.0,Corporate Headquarters,Armonk,10504,41.109534,-73.723999
7210974,4521,c:3246,NaN,USD,2006-09-06,Michael Lipton,Computer Science & Engineering,False,Company,3246,ibm,software,ipo,NaN,NaN,http://www.ibm.com,USA,NY,Armonk,New York,2000-08-28,2008-03-25,2.0,2.0,NaN,NaN,NaN,NaN,870.0,Corporate Headquarters,Armonk,10504,41.109534,-73.723999
7210975,4521,c:3246,NaN,USD,2006-09-06,Blair Hankins,Mathematics,False,Company,3246,ibm,software,ipo,NaN,NaN,http://www.ibm.com,USA,NY,Armonk,New York,2000-08-28,2008-03-25,2.0,2.0,NaN,NaN,NaN,NaN,870.0,Corporate Headquarters,Armonk,10504,41.109534,-73.723999
7210976,4521,c:3246,NaN,USD,2006-09-06,Shanker Ramamurthy,Innformation Science,False,Company,3246,ibm,software,ipo,NaN,NaN,http://www.ibm.com,USA,NY,Armonk,New York,2000-08-28,2008-03-25,2.0,2.0,NaN,NaN,NaN,NaN,870.0,Corporate Headquarters,Armonk,10504,41.109534,-73.723999


In [9]:
# print all columns and their unique values
for c in df.columns:
    print(c + " - " + str(df[c].unique()))

acquisition_id - [ 209  263  273 ... 4519 4520 4521]
relationship_object_id - ['c:59' 'c:24' 'c:212' 'c:227' 'c:5' 'c:318' 'c:20' 'c:342' 'c:349'
 'c:355' 'c:351' 'c:307' 'c:29' 'c:541' 'c:568' 'c:570' 'c:317' 'c:583'
 'c:584' 'c:590' 'c:25' 'c:622' 'c:13171' 'c:665' 'c:674' 'c:694' 'c:156'
 'c:754' 'c:823' 'c:826' 'c:838' 'c:840' 'c:845' 'c:851' 'c:988' 'c:1035'
 'c:45' 'c:1246' 'c:489' 'c:1335' 'c:1354' 'c:1081' 'c:396' 'c:350'
 'c:1242' 'c:1727' 'c:1438' 'c:1504' 'c:608' 'c:1735' 'c:2042' 'c:2050'
 'c:2272' 'c:2312' 'c:2358' 'c:2381' 'c:2602' 'c:2720' 'c:2758' 'c:2778'
 'c:191' 'c:2175' 'c:760' 'c:2881' 'c:966' 'c:1873' 'c:3063' 'c:3104'
 'c:3165' 'c:3253' 'c:3316' 'c:2537' 'c:3619' 'c:757' 'c:4010' 'c:4199'
 'c:4247' 'c:308' 'c:4272' 'c:3754' 'c:304' 'c:4797' 'c:4818' 'c:2680'
 'c:4845' 'c:46' 'c:424' 'c:5027' 'c:5059' 'c:5160' 'c:5396' 'c:4175'
 'c:5471' 'c:1895' 'c:5546' 'c:5548' 'c:5564' 'c:5622' 'c:250' 'c:1663'
 'c:3526' 'c:1654' 'c:475' 'c:5744' 'c:5807' 'c:5837' 'c:1926' 'c:

status - ['ipo' 'operating' 'closed']
founded_at - ['1998-09-07' '1986-01-01' '1994-01-01' '1984-03-19' '2004-02-01'
 '2007-07-01' '1995-09-01' '1923-10-16' '1995-01-01' '1939-01-01'
 '1985-05-24' '2007-08-01' '1922-01-01' '1999-01-01' nan '2000-09-20'
 '1971-01-01' '1981-01-01' '1996-01-01' '2000-01-01' '2001-01-01'
 '2006-10-01' '1910-01-01' '1997-02-07' '1998-01-01' '1982-01-01'
 '1980-01-01' '2005-09-01' '1906-01-01' '1985-06-17' '1995-08-01'
 '1974-04-04' '1992-01-01' '1999-09-01' '2004-01-01' '2005-06-01'
 '2005-01-15' '2005-02-01' '2005-01-01' '2002-01-01' '1984-01-01'
 '1997-01-01' '2007-01-01' '2006-06-01' '1998-01-21' '1979-01-01'
 '1980-09-19' '1983-01-01' '2001-02-07' '1902-01-01' '2006-01-01'
 '1946-01-01' '1990-01-01' '1977-01-01' '2003-10-01' '1998-08-01'
 '1996-06-16' '2006-12-01' '1963-01-01' '1997-09-23' '1976-04-01'
 '2008-01-01' '2001-08-01' '2002-08-01' '1999-08-01' '2003-01-01'
 '2002-01-17' '1997-10-01' '1987-10-14' '1997-05-01' '2003-09-01'
 '2006-04-01' '2007-0

 'Fairfield' 'Salt Lake City' 'Raleigh-Durham' 'Berlin' 'Hamilton']
first_investment_at - ['2004-06-18' '2006-10-01' nan '2008-04-03' '2008-12-01' '2007-10-23'
 '2004-01-22' '2009-04-30' '2010-03-08' '2006-03-01' '1996-01-01'
 '1998-01-01' '2013-02-01' '2010-11-08' '2005-04-01' '2006-06-01'
 '2006-12-01' '2005-08-01' '2008-03-03' '2004-01-01' '2011-09-08'
 '2006-10-30' '2009-12-14' '2005-07-06' '2008-01-01' '2008-04-24'
 '2007-01-01' '2005-11-01' '1987-06-16' '2005-04-17' '2008-09-03'
 '2008-02-29' '2013-03-27' '2007-06-05' '2013-06-27' '2002-09-16'
 '2011-02-17' '2006-04-01' '2007-06-18' '2008-08-06' '2010-09-27'
 '2010-05-01' '2009-10-20' '1995-11-30' '2010-01-11' '2005-04-14'
 '2008-06-01' '2005-05-05' '1998-05-01' '2007-09-10' '2009-06-05'
 '2010-06-28' '2006-02-15' '2004-07-26' '2003-11-17' '2013-07-16'
 '2007-03-21' '2012-08-20' '2013-03-07' '2007-03-22' '2009-09-30'
 '2010-04-07' '2000-08-28']
last_investment_at - ['2013-05-01' '2007-02-26' nan '2010-02-18' '2013-11-12' '2013-09

 'Company Headquarters']
city_y - ['Mexico' 'Haifa' 'Tel-Aviv' 'Istanbul' 'Dubai' 'Mountain View'
 'Ann Arbor' 'Atlanta' 'Austin' 'Boulder' 'Cambridge' 'Chicago' 'Coppell'
 'Dallas' 'Denver' 'Birmingham' 'Irvine' 'Kirkland' 'New York' 'Tempe'
 'Pittsburgh' 'San Francisco' 'Santa Monica' 'Seattle' 'Washington'
 'Melbourne' 'Sydney' 'Beijing' 'Guangzhou' 'Shanghai' 'Central'
 'Bangalore' 'Gurgaon' 'New Delhi' 'Hyderabad' 'Mumbai' 'Tokyo' 'Osaka'
 'Seoul' 'Singapore' 'Taipei' nan 'Helsinki' 'Paris' 'Hamburg' 'Dublin'
 'Milan' 'Amsterdam' 'Oslo' 'Trondheim' 'Kraków' 'Wroclaw' 'Moscow'
 'St. Petersburg' 'Madrid' 'Luleå' 'Stockholm' 'Zürich' 'London'
 'Manchester' 'Montreal' 'Toronto' 'Waterloo' 'Buenos Aires' 'Sao Paulo'
 'Knoxville' 'Menlo Park' 'San Jose' 'Burbank' 'West Hollywood'
 'Palo Alto' 'Sunnyvale' 'Newton' 'Cincinnati' 'Woodland Hills'
 'Unterhaching' 'Redwood City' 'Englewood' 'Secaucus' 'Portland' 'Redmond'
 'Boise' 'Kansas City' 'Fort Myers' 'Covent Garden, London' 'Munich'
 '

From the above result we deduce the following:
 * we don't need the columns which only have IDs in our ML model: acquisition_id, relationship_object_id, entity_id
 * investigate more the columns which contain nan values: price_amount, price_currency_code, subject, first_investment_at, last_investment_at, investment_rounds, invested_companies, first_funding_at, last_funding_at, funding_rounds, funding_total_usd, zip_code  
 * the column 'Aquired' only contains the value _False_ => we don't need it
 * the column 'entity_type'  only contains the value _Company_ => we don't need it
 * the column 'closed\_at' contains only 4 unique values and nan => further investigate 
 * the columns 'latitude' and 'longitude' are not needed since we have information about location, such as contry, city and zip code
 * the column 'relationships' contains irrelevant data for us => we drop it

In [10]:
# drop the columns which contains ids
df.drop(['acquisition_id', 'relationship_object_id', 'entity_id'], axis=1, inplace=True)

# drop the columns which we don't need
df.drop(['Aquired', 'entity_type', 'relationships', 'latitude', 'longitude'], axis=1, inplace=True)

In [11]:
# Summary statistics
df.describe(include="all")

,price_amount,price_currency_code,acquired_at,full_name,subject,normalized_name,category_code,status,founded_at,closed_at,homepage_url,country_code,state_code_x,city_x,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,description_y,city_y,zip_code
count,1.995730e+06,7210949,7210707,7210715,5864970,7210978,7210800,7210978,7151879,25,7210591,7210978,7200134,7204097,7210978,7187982,7187982,7.187982e+06,7.187982e+06,6704149,6704149,6.704149e+06,6.704113e+06,6994072,7011070,6587020
unique,NaN,6,1414,8937,2361,184,21,3,93,4,181,23,21,100,59,62,63,NaN,NaN,68,76,NaN,NaN,201,208,305
top,NaN,USD,2009-11-09,Erick Tseng,Computer Science,google,search,ipo,1998-09-07,2009-12-23,http://google.com,USA,CA,Mountain View,SF Bay,2004-06-18,2013-05-01,NaN,NaN,1998-08-01,1999-06-07,NaN,NaN,Google Seattle,Seattle,98103
freq,NaN,7104820,104242,8784,1198520,6474215,6475198,7202329,6474215,10,6474215,7201046,6845495,6483703,6830840,6474215,6518695,NaN,NaN,6474215,6474215,NaN,NaN,193260,202671,193260
mean,6.241055e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.787791e+01,2.493463e+01,NaN,NaN,2.018723e+00,3.645655e+07,NaN,NaN,NaN
std,2.138176e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.477186e+00,7.117151e+00,NaN,NaN,5.745654e-01,1.529393e+08,NaN,NaN,NaN
min,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1.000000e+00,NaN,NaN,1.000000e+00,1.000000e+06,NaN,NaN,NaN
25%,2.800000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.900000e+01,2.600000e+01,NaN,NaN,2.000000e+00,2.510000e+07,NaN,NaN,NaN
50%,7.000000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.900000e+01,2.600000e+01,NaN,NaN,2.000000e+00,2.510000e+07,NaN,NaN,NaN
75%,3.370000e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.900000e+01,2.600000e+01,NaN,NaN,2.000000e+00,2.510000e+07,NaN,NaN,NaN


From the dataset statistics we notice in the 'count' row that the column 'closed_at' only has 25 values that are not NaN, thus, we drop the 'closed_at' column too.

In [13]:
print("Column 'closed_at' has {} cells containing nan values out of the total {} number of cells".format(
    get_nan_values_for_column(df, 'closed_at'),
    df.shape[0])
)

Column 'closed_at' has 7210953 cells containing nan values out of the total 7210978 number of cells


In [14]:
# drop 'closed_at' column
df.drop(['closed_at'], axis=1, inplace=True)

We want our Machine Learning model to predict the status ('ipo', 'operating' or 'closed') of companies. For this, we deduced that the relevant columns, from the dataframe, that could be used as features are: **subject, category_code, founded_at, funding_total_usd and city_y**, and the column **status** will represent the label of the data.

In [15]:
# drop irrelevant columns from the dataframe
df.drop(['price_amount', 'price_currency_code', 'acquired_at', 'full_name',
       'normalized_name','homepage_url', 'country_code', 'state_code_x', 'city_x', 'region',
       'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'first_funding_at', 'last_funding_at',
       'funding_rounds', 'description_y', 'zip_code'], 
        axis=1, inplace=True)

In [16]:
# Summary statistics for the reduced data frame
df.describe(include="all")

,subject,category_code,status,founded_at,funding_total_usd,city_y
count,5864970,7210800,7210978,7151879,6.704113e+06,7011070
unique,2361,21,3,93,NaN,208
top,Computer Science,search,ipo,1998-09-07,NaN,Seattle
freq,1198520,6475198,7202329,6474215,NaN,202671
mean,NaN,NaN,NaN,NaN,3.645655e+07,NaN
std,NaN,NaN,NaN,NaN,1.529393e+08,NaN
min,NaN,NaN,NaN,NaN,1.000000e+06,NaN
25%,NaN,NaN,NaN,NaN,2.510000e+07,NaN
50%,NaN,NaN,NaN,NaN,2.510000e+07,NaN
75%,NaN,NaN,NaN,NaN,2.510000e+07,NaN


The total number of rows of the dataframe is 7210798, but if we inspect the _count_ rows of the statistics we notice that the other columns have less rows, e.g. the column subject only has 5864970. This means that the rest of the values, up to 7210798 are NaN  values.    
   
Next, we will delete the rows that contain NaN values, since those rows will impact negatively our model.

In [17]:
df = drop_nan_values_from_columns(df, ['subject','category_code','status', 'founded_at', 'funding_total_usd', 'city_y'])
df

,subject,category_code,status,founded_at,funding_total_usd,city_y
0,Computer Science,search,ipo,1998-09-07,25100000.0,Mountain View
1,Computer Science,search,ipo,1998-09-07,25100000.0,Ann Arbor
2,Computer Science,search,ipo,1998-09-07,25100000.0,Atlanta
3,Computer Science,search,ipo,1998-09-07,25100000.0,Austin
4,Computer Science,search,ipo,1998-09-07,25100000.0,Boulder
...,...,...,...,...,...,...
5296259,"Finance, Economics, Organizational Psychology",advertising,operating,2006-11-01,53157047.0,Emeryville
5296260,Industrial Engineering,advertising,operating,2006-11-01,53157047.0,Emeryville
5296261,General Management,advertising,operating,2006-11-01,53157047.0,Emeryville
5296262,Communications,advertising,operating,2006-11-01,53157047.0,Emeryville


In [18]:
# check to see if rows have nan values
are_columns_with_nan_values = False

for c in df.columns:
    if 0 != get_nan_values_for_column(df, c):
        are_columns_with_nan_values = True
        print(f"Column {c} still has NaN values")

if not are_columns_with_nan_values:
    print("No column has NaN values")

No column has NaN values


In [29]:
print(f"The maximum value from the 'funding_total_usd' column is " + str(df['funding_total_usd'].loc[df['funding_total_usd'].idxmax()]))

The maximum value from the 'funding_total_usd' column is 2425700000.0


Since the maximum value on the 'funding_total_usd' column is 2425700000 we can change its data type from float64 to float32. (we had a MemoryError when testing the pipeline and now we try to save space from where we can)

In [37]:
df['funding_total_usd'] = df['funding_total_usd'].astype(np.float32)

In [32]:
df.dtypes

subject              object
category_code        object
status               object
founded_at           object
funding_total_usd     int32
city_y               object
dtype: object

At the moment, the data in the dataframe there might be some ordering, i.e., rows belonging to the same subject or  category_code might be grouped together. We don't want our model to learn this ordering, so we need to make the dataset as random as possible. For this, we will shuffle the data inside the dataframe, i.e., we will randomize the order of the rows.

In [33]:
# Shuffle
df = df.take(np.random.permutation(len(df)))
df.reset_index(drop=True, inplace=True)
df.head(5)

,subject,category_code,status,founded_at,funding_total_usd,city_y
0,Business,search,ipo,1998-09-07,25100000,Trondheim
1,Journalism,advertising,operating,1987-10-14,86400002,New York
2,Public Policy and Computer Science,search,ipo,1998-09-07,25100000,St. Petersburg
3,Mechanical Engineering,search,ipo,1998-09-07,25100000,Milan
4,"Economics, PreMed",search,ipo,1998-09-07,25100000,Santa Monica


In [57]:
# due to the MemoryError received when running cross_val_score function, we create a smaller data frame that has
# 1000000 initial rows
df_smaller = df.take(np.random.permutation(len(df))[:1000000])
df_smaller.reset_index(drop=True, inplace=True)
df_smaller.head(5)

,subject,category_code,status,founded_at,funding_total_usd,city_y
0,Computer Science,search,ipo,1998-09-07,25100000.0,Atlanta
1,"Computer Science,Economics",search,ipo,1998-09-07,25100000.0,Montreal
2,psychology,search,ipo,1998-09-07,25100000.0,Bangalore
3,Finance,search,ipo,1998-09-07,25100000.0,Bangalore
4,"Business Administration and Management, General",search,ipo,1998-09-07,25100000.0,Atlanta


In [58]:
# The features we want to select
numeric_features = ["funding_total_usd"]
nominal_features = ["subject", "category_code", "founded_at", "city_y"]

numeric_pipeline = Pipeline([
        ("selector", DataFrameSelector(numeric_features)),
        ("scaler", StandardScaler())
    ])

nominal_pipeline = Pipeline([
        ("selector", DataFrameSelector(nominal_features)), 
        ("binarizer", FeatureBinarizer([df[feature].unique() for feature in nominal_features]))])


pipeline = Pipeline([("union", FeatureUnion([("numeric_pipeline", numeric_pipeline), 
                                            ("nominal_pipeline", nominal_pipeline)])),
                              ("estimator", LogisticRegression(multi_class="multinomial", solver="newton-cg"))])

In [59]:
# get the labels
y = df_smaller['status'].values
y

array(['ipo', 'ipo', 'ipo', ..., 'ipo', 'ipo', 'ipo'], dtype=object)

In [60]:
# Stratified k-fold cross-validation
kf = StratifiedKFold(n_splits = 10)
np.mean(cross_val_score(pipeline, df_smaller, y, scoring="accuracy", cv=kf))

C:\Users\asabau\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:597: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.9999930000299996